In [ ]:
# import packages
%matplotlib widget 
import os 
import numpy as np
import pandas as pd
import geopandas as gpd
from cartopy.feature import ShapelyFeature
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import shapely.geometry 
from shapely import Point, Polygon
from pyproj import CRS
import seaborn as sns
plt.ion()

In [ ]:
# function to assign and project correct CRS
def set_and_reproj(gdf, current_epsg=4326, target_epsg=32629): # defines function name and parameters
    '''
    Assigns current epsg to a GeoDataFrame if missing, then reprojects it to target CRS.

    Parameters:
    -----------
    - gdf: GeoDataFrame
    - current_epsg: EPSG code of current CRS (default: 4326)
    - target_epsg: EPSG code to reproject to (default: 32629)

    Returns:
    --------
    - Reprojected GeoDataFrame
    '''
    if gdf.crs is None: # if geodataframe as no crs
        gdf = gdf.set_crs(epsg=current_epsg) # set the crs to current_epsg (4326)
        
    return gdf.to_crs(epsg=target_epsg) # return the geodataframe witrh target epsg (32629)
    

In [ ]:
# read in files and data using set_and_reproj function to assign crs
outline = set_and_reproj(gpd.read_file('data_files/NI_outline.shp'), current_epsg=4326)
towns = set_and_reproj(gpd.read_file('data_files/Towns.shp'), current_epsg=4326)
counties = set_and_reproj(gpd.read_file('data_files/Counties.shp'), current_epsg=4326)
water = set_and_reproj(gpd.read_file('data_files/Water.shp'), current_epsg=4326)
aonb = set_and_reproj(gpd.read_file('data_files/AONB.shp'), current_epsg=4326)
assi = set_and_reproj(gpd.read_file('data_files/ASSI.shp'), current_epsg=4326)

# read in NBN csv file
lepi = pd.read_csv('data_files/egm722_project_data_NBN.csv')

In [ ]:
# define generate handles function
def generate_handles(labels, colors, edge='k', alpha=1): # defines function name and parameters
    
    '''
    Generate matplotlib handles to create a legend for each feature on the map.

    Parameters
    ----------

    labels: list(str) - text labels of features to show on legend.
    colors: list(matplotlib color) - colors used for each feature on map.
    edge: matplotlib color (default: 'k') - color to use for edge of legend patches
    alpha: float (default: 1.0) - alpha value to use for legend patches

    Returns
    -------

    handles: list(matplotlib.patches.Rectangle) - list of legend patches to pass to ax.legend()

    '''
    lc = len(colors) # gets length of colour list
    handles = [] # create an empty list
    for ii in range(len(labels)): # for each label/colour pair, make an empty box for legend
        handles.append(mpatches.Rectangle((0, 0), 1, 1, facecolor=colors[ii % lc], edgecolor = edge, alpha=alpha))
    return handles

In [ ]:
# create function to add scale bar to map
def scale_bar(ax, length=20, location=(0.92, 0.95)): # define function name and parameters
    """
    Create a scale bar in a cartopy GeoAxes.

    Parameters
    ----------

    ax : cartopy.mpl.geoaxes.GeoAxes
        the cartopy GeoAxes to add the scalebar to.

    length : int, float (default 20)
        the length of the scalebar, in km

    location : tuple(float, float) (default (0.92, 0.95))
        the location of the center right corner of the scalebar, in fractions of the axis.

    Returns
    -------
    ax : cartopy.mpl.geoaxes.GeoAxes
        the cartopy GeoAxes object

    """
    x0, x1, y0, y1 = ax.get_extent() # get the current extent of the axis
    sbx = x0 + (x1 - x0) * location[0] # get the right x coordinate of the scale bar
    sby = y0 + (y1 - y0) * location[1] # get the right y coordinate of the scale bar

    ax.plot([sbx, sbx-length*1000], [sby, sby], color='k', linewidth=4, transform=ax.projection) # plot a thick black line
    ax.plot([sbx-(length/2)*1000, sbx-length*1000], [sby, sby], color='w', linewidth=2, transform=ax.projection) # plot a white line from 0 to halfway

    ax.text(sbx, sby-(length/4)*1000, f"{length} km", ha='center', transform=ax.projection, fontsize=6) # add a label at the right side
    ax.text(sbx-(length/2)*1000, sby-(length/4)*1000, f"{int(length/2)} km", ha='center', transform=ax.projection, fontsize=6) # add a label in the center
    ax.text(sbx-length*1000, sby-(length/4)*1000, '0 km', ha='center', transform=ax.projection, fontsize=6) # add a label at the left side

    return ax

In [ ]:
# create function to compute Simpson's Diversity Index
def simpsons_diversity(abundances):
    """
    Calculate Simpson's Diversity indices from species abundances.

    Parameters:
    -----------
    abundances : array-like
        List or array of species abundances

    Returns:
    --------
    dict : Dictionary containing D, 1-D, and 1/D
    """
    abundances = np.array(abundances) 
    total = abundances.sum() 
    proportions = abundances / total
    D = np.sum(proportions ** 2)

    return {
        'D': D,
        '1-D': 1 - D,
        '1/D': 1 / D
    }

# function copied from The Research Scientist Pod, Suf (2024) 

In [ ]:
# convert the lepi DataFrame to a GeoDataFrame
lepi['geometry'] = list(zip(lepi['Longitude (WGS84)'], lepi['Latitude (WGS84)'])) # create geometry column using lat/long columns and zip together
lepi['geometry'] = lepi['geometry'].apply(Point) # turn the geometry column into points

del lepi['Data provider'], lepi['Common name'], lepi['Locality'] # remove unwanted columns
lepi = lepi.rename(columns={'Scientific name': 'sci_name', 'Start date': 'date', 'Longitude (WGS84)': 'longitude', 'Latitude (WGS84)': 'latitude'}) # rename columns
lepi = gpd.GeoDataFrame(lepi, geometry='geometry') # create the new GeoDataFrame

lepi = set_and_reproj(lepi) # use set and reproject function to modify crs

lepi.to_file('lepi.shp') # save to shapefile

In [ ]:
ni_utm = ccrs.UTM(29) # create lat/long reference system to transform data

In [ ]:
# create map 1 object and axis, extent and NI outline
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'projection': ni_utm})  # create a figure 8x8 (page size in inches) and an axis object in the figure using utm projection

xmin, ymin, xmax, ymax = outline.total_bounds # using boundary of outline features, zooms map to NI
ax.set_extent([xmin-5000, xmax+5000, ymin-5000, ymax+5000], crs=ni_utm) # reordered coordinates for set_extent

# add NI outline to map
outline_feature = ShapelyFeature(outline['geometry'], ni_utm, edgecolor='k', facecolor='w') 

outline['area_km2'] = outline.geometry.area / 1e6 # add and calculate area in square km from geometry column
del outline['area'] # delete old area column

ax.add_feature(outline_feature)

# add counties to map 1 using separate colours
num_counties = len(counties.CountyName.unique()) # get the number of unique counties in the dataset

county_colors = ['#665191','#a05195','#d45087','#f95d6a','#ff7c43','#ffa600'] # pick colours for individual counties

county_names = list(counties.CountyName.unique()) # get a list of unique names for the county boundaries
county_names.sort() # sort counties alphabetically by name

for ii, name in enumerate(county_names):
    feat = ShapelyFeature(counties.loc[counties['CountyName'] == name, 'geometry'], # first argument is the geometry
                          ccrs.CRS(counties.crs), # second argument is the CRS
                          edgecolor='k', # outline in black
                          facecolor=county_colors[ii], # set face color to corresponding color from the list
                          linewidth=1, # set outline width as 1 pt
                          alpha=0.2) # set alpha (transparency) to be 0.2 (out of 1)
    ax.add_feature(feat) # add feature to map 

# update county_names to take out of uppercase text using .title()
nice_names = []  # initalize an empty list
for name in county_names: # iterate through county names
    nice_names.append(name.title()) # change name to title case


# add other features 
# add assi to map
assi_feat = ShapelyFeature(assi['geometry'],
                      ni_utm,
                      edgecolor='k',
                      facecolor='lawngreen',
                      linewidth=0.7)
ax.add_feature(assi_feat)

# add aonb to map
aonb_feat = ShapelyFeature(aonb['geometry'],
                      ni_utm,
                      edgecolor='k',
                      facecolor='green',
                      linewidth=0.7,
                     alpha=0.4)
ax.add_feature(aonb_feat)

# add the water features with single colour symbology
lakes_feat  = ShapelyFeature(water['geometry'],
                            ni_utm,
                            edgecolor='royalblue',
                            facecolor='royalblue', 
                            linewidth=1)
ax.add_feature(lakes_feat)

# add towns
towns_handle = ax.plot(towns.geometry.x, towns.geometry.y, 'o', color='k', ms=3)

# add lepi points
lepi_handle = ax.plot(lepi.geometry.x, lepi.geometry.y, 'o', color='#fc4e2a', ms=0.5)

# add map_01 furniture
# generate a list of handles for the dataframes, add list of names, colors, and set transparency
county_handle = generate_handles(counties.CountyName.unique(), county_colors, alpha=0.4)
aonb_handle = generate_handles(['aonb'], ['green'], alpha=0.4)
assi_handle = generate_handles(['assi'], ['lawngreen'])
water_handle = generate_handles(['lakes'], ['royalblue'])

# adds legend to map, using handles and labels corresponding to the features
handles = county_handle + water_handle + assi_handle + aonb_handle + towns_handle + lepi_handle # use '+' to concatenate (combine) lists
labels = nice_names + ['Lakes', 'ASSI', 'AONB', 'Towns', 'Species Occurrence'] # legend labels

leg = ax.legend(handles, labels, title='Legend', title_fontsize=10, # adds the legend, including title, font and location parameters
                 fontsize=8, loc='upper left', frameon=True, framealpha=1)

# adds gridlines with labels at 0.5 degree intervals
gridlines = ax.gridlines(draw_labels=True, # draw  labels for the grid lines
                         xlocs=[-8, -7.5, -7, -6.5, -6, -5.5], # add longitude lines at 0.5 deg intervals
                         ylocs=[54, 54.5, 55, 55.5]) # add latitude lines at 0.5 deg intervals
gridlines.left_labels = False # turn off the left-side labels
gridlines.bottom_labels = False # turn off the bottom labels

scale_bar(ax) # place the scale bar from function above, in upper right corner of the map

# add town name labels 
for ind, row in towns.iterrows(): # towns.iterrows() returns the index and row
    x, y = row.geometry.x, row.geometry.y # get the x,y location for each town
    ax.text(x, y, row['TOWN_NAME'].title(), fontsize=7) # use plt.text to place a label at x,y

fig.savefig('map_01.png', bbox_inches='tight', dpi=300) # saves map 1

In [ ]:
# create a kernel density plot using seaborn 
# ensure lepi is a gdf with geometry, then reproject
lepi = lepi.set_geometry(gpd.points_from_xy(lepi.longitude, lepi.latitude), crs='EPSG:4326')
lepi_utm = lepi.to_crs(ni_utm) # reproject lepi to match kde map projection

# create map 2
kde_fig, kde_ax = plt.subplots(figsize=(8, 8), subplot_kw={'projection': ni_utm})  # create a figure 8x8 (page size in inches) and an axis object in the figure using utm projection
xmin, ymin, xmax, ymax = outline.total_bounds # using boundary of outline features, zooms map to NI
kde_ax.set_extent([xmin-5000, xmax+5000, ymin-5000, ymax+5000], crs=ni_utm) # reordered coordinates for set_extent

# add NI outline to map
outline_feature = ShapelyFeature(outline['geometry'], ni_utm, facecolor='#f2f2f2', edgecolor='black', linewidth=0.8 ) 
kde_ax.add_feature(outline_feature, zorder=1) # puts at bottom of image

# add lakes
lakes_feat  = ShapelyFeature(water['geometry'], ni_utm, edgecolor='royalblue',
                            facecolor='royalblue', alpha=0.3, linewidth=0.1)
kde_ax.add_feature(lakes_feat, zorder=1)

# add towns
towns_handle = kde_ax.plot(towns.geometry.x, towns.geometry.y, 'o', color='k', ms=2.5) 

# colour bar keyword arguments
cbar_kws = {'label': 'Diversity', 'ticks': [0, 1]} # Miradulo (2017)


# make the kde plot
sns.kdeplot( 
    data=lepi_utm, # using lepi dataframe
    x=lepi_utm.geometry.x,
    y=lepi_utm.geometry.y,
    fill=True,
    cmap='PiYG', # colour ramp
    cbar=True, # add colour bar
    cbar_kws = cbar_kws,
    alpha=0.4, # transparency
    legend=True, # add legend
    gridsize=200,
    levels=20,
    zorder=2,
    ax=kde_ax) # ensures stays top layer of image
# sash_wash (2021)

#cbar.set_ticks([0, 1], labels=['Low', 'High']), # colour bar ticks and labels (Matlpotlib, 2025)

# add map 2 furniture
# add town name labels 
for ind, row in towns.iterrows(): # towns.iterrows() returns the index and row
    x, y = row.geometry.x, row.geometry.y # get the x,y location for each town
    kde_ax.text(x, y, row['TOWN_NAME'].title(), fontsize=7) # use plt.text to place a label at x,y

scale_bar(kde_ax) # place the scale bar from function above, in upper right corner of the map

# adds gridlines with labels at 0.5 degree intervals
kde_gridlines = kde_ax.gridlines(draw_labels=True, # draw  labels for the grid lines
                         xlocs=[-8, -7.5, -7, -6.5, -6, -5.5], # add longitude lines at 0.5 deg intervals
                         ylocs=[54, 54.5, 55, 55.5]) # add latitude lines at 0.5 deg intervals
kde_gridlines.left_labels = False # turn off the left-side labels
kde_gridlines.bottom_labels = False # turn off the bottom labels

plt.show()

kde_fig.savefig('map_02.png', bbox_inches='tight', dpi=300) # saves map 2


In [ ]:
# Diversity analysis 
# how many unique species there are in Northern Ireland
richness = lepi['sci_name'].nunique() # counts the number of unique species

# Simpson's Biodiversity Index for Northern Ireland
abundances = lepi['sci_name'].value_counts().values  # the lepidoptera example
results = simpsons_diversity(abundances)

print(richness)
print(f"Simpson's Dominance (D): {results['D']:.3f}") # prints dominance to 3 d.p.
print(f"Simpson's Diversity (1-D): {results['1-D']:.3f}") # prints diversity to 3 d.p.
print(f"Simpson's Reciprocal (1/D): {results['1/D']:.3f}") # prints reciprocal to 3 d.p.

# Suf (2024)

In [ ]:
# join counties and lepi dataframes and do diversity by county
# match crs to counties
county_lepi = lepi.to_crs(counties.crs)
# spatially join lepi points to counties
lepi_join_counties = gpd.sjoin(county_lepi, counties[['CountyName', 'geometry']], how='inner', lsuffix='left', rsuffix='right')

# compute species richness by county
SR_counties = lepi_join_counties.groupby(['CountyName']) # group the counties by name
SRC = SR_counties['sci_name'].nunique() # count the number of species per county
print(SRC)

# compute diversity by county and save as new dataframe
county_results = [] # make new list
for county, df in lepi_join_counties.groupby('CountyName'): # iterate over county rows grouped by county name
    county_abundance = df['sci_name'].value_counts().values # counts how many of each scientific name
    ca_results = simpsons_diversity(county_abundance) # applying Simpson's index function to abundance counts
    county_results.append({ # add results to a new dataframe with following columns
        'CountyName': county,
        'Simpsons_D': ca_results['D'],
        'Simpsons_1-D': ca_results['1-D'],
        'Simpsons_1/D': ca_results['1/D']
    })
county_div = pd.DataFrame(county_results) # save as new dataframe

# check counties and the county_div dataframes have same case field
counties['CountyName'] = counties['CountyName'].str.title().str.strip() # ensure county names are in titlecase
county_div['CountyName'] = county_div['CountyName'].str.title().str.strip() # ensure county names are in titlecase

# merge the county_div and counties, to include geometry in county_div
county_div_geo = counties.merge(county_div, on='CountyName')

county_div_geo

In [ ]:
# create map 3 to show Simpson's reciprocal (1/D) by county
county_SDI_fig, county_SDI_ax = plt.subplots(figsize=(8, 8), subplot_kw={'projection': ni_utm})  # create a figure 8x8 (page size in inches) and an axis object in the figure using utm projection

xmin, ymin, xmax, ymax = counties.total_bounds # using boundary of counties features, zooms map to NI
county_SDI_ax.set_extent([xmin-5000, xmax+5000, ymin-5000, ymax+5000], crs=ni_utm) # reordered coordinates for set_extent

# plot the reciprocal index (1/D) by county
county_div_geo.plot(
    column='Simpsons_1/D', # use reciprocal index
    cmap='viridis', # colour ramp
    vmin=130, # minimum value
    vmax=170, # maximum value
    legend=True, # side bar
    edgecolor='black',
    linewidth=0.8, 
    ax=county_SDI_ax)

# add map furniture
scale_bar(county_SDI_ax) # place the scale bar from function above, in upper right corner of the map

# adds gridlines with labels at 0.5 degree intervals
sdi_gridlines = county_SDI_ax.gridlines(draw_labels=True, # draw  labels for the grid lines
                         xlocs=[-8, -7.5, -7, -6.5, -6, -5.5], # add longitude lines at 0.5 deg intervals
                         ylocs=[54, 54.5, 55, 55.5]) # add latitude lines at 0.5 deg intervals
sdi_gridlines.left_labels = False # turn off the left-side labels
sdi_gridlines.bottom_labels = False # turn off the bottom labels

# add county name labels
for i, row in county_div_geo.iterrows(): # iterate through rows
    centroid = row.geometry.centroid # find centroid of county polygons
    county_SDI_ax.text( # add text
        centroid.x, # x loc
        centroid.y, # y loc
        row['CountyName'], # find county names
        fontsize=8, 
        color='w',
        ha='center', # label position
        va='center',
        transform=ni_utm,
        zorder=5 # put labels on top
    )

county_SDI_fig.savefig('map_03.png', bbox_inches='tight', dpi=300) # saves the map

## References
- Matplotlib (2025) Available at: [https://matplotlib.org/stable/api/colorbar_api.html]
- Miradulo (2017) Answer 132. *Stackoverflow*. Available at: [https://stackoverflow.com/questions/42092218/how-to-add-a-label-to-seaborn-heatmap-color-bar#:~:text=You%20could%20set%20it%20afterwards%20after%20collecting%20it,the%20colorbar%20such%20as%20tick%20frequency%20or%20formatting]
- Sash_wash (2021) Available at: [https://gis.stackexchange.com/questions/394154/using-geometry-points-to-get-data-in-order-to-plot-heatmap/395039#395039]
- Suf (2024) Simpson's Diversity Index: Calculating Species Dominance and Evenness. *The Research Scientist Pod*. Available at: [https://researchdatapod.com/simpsons-diversity-index/](https://researchdatapod.com/simpsons-diversity-index/) (Accessed: 25 April 2025).